# Initialize

In [1]:
# Imports

import os
import rasterio
import geopandas as gpd
from rasterio.features import geometry_mask
from sklearn.metrics import cohen_kappa_score

In [2]:
# Get and print the current working directory
current_dir = os.getcwd()
print('Current Working Directory:', current_dir)

# Update and print current working directory
temp_dirpath = os.path.join('D:\\', 'Akshaya')
os.chdir(temp_dirpath)
updated_current_dir = os.getcwd()
print("Updated Current Working Directory:", updated_current_dir)

# Set and print the raw DATA directory
rawDATA_dir = os.path.join(updated_current_dir, 'DATA')
print('Raw DATA Directory:', rawDATA_dir)

# Set and print the Output directory
output_dir = os.path.join(updated_current_dir, 'LULC_dataset', 'jupyterNB_outputs')
print('Output Directory:', output_dir) 

Current Working Directory: D:\Akshaya\LULC_dataset
Updated Current Working Directory: D:\Akshaya
Raw DATA Directory: D:\Akshaya\DATA
Output Directory: D:\Akshaya\LULC_dataset\jupyterNB_outputs


In [3]:
# Funtion to create a new folder at specified folder path
def create_folder(folder_path, folder_name):
    try:
        # Join folder path and folder name to create the full path
        full_path = os.path.join(folder_path, folder_name)
        
        # Check if the folder already exists
        if not os.path.exists(full_path):
            # Create the folder
            os.makedirs(full_path)
            print(f"Folder '{folder_name}' created successfully at '{folder_path}'.")
        else:
            print(f"Folder '{folder_name}' already exists at '{folder_path}'.")
    except Exception as e:
        print(f"An error occurred: {str(e)}")     
    return full_path
        
# Create and print Output directory for the current jupyterNB
current_output_dir = create_folder(folder_path=output_dir, folder_name='kappa_index')
print('Current jupyterNB Output Directory:', current_output_dir)

Folder 'kappa_index' already exists at 'D:\Akshaya\LULC_dataset\jupyterNB_outputs'.
Current jupyterNB Output Directory: D:\Akshaya\LULC_dataset\jupyterNB_outputs\kappa_index


# kappa score 2019

In [4]:
# Load the shapefile
shapefile_path = os.path.join(rawDATA_dir, 'DL2', 'extract_2019.shp')
gdf = gpd.read_file(shapefile_path)
print(gdf.shape)
gdf.head()

(120543, 20)


,elevation,B11,B12,NDWI,NDVI,sample,B2,B3,B4,B5,B6,B7,NDMI,B8,classvalue,NDBI,EVI,NDBaI,NBR,geometry
0,47,2891.0,2589.0,-0.233540,0.112605,train,990.0,1234.0,1584.0,1742.0,1840.0,2005.0,-0.185565,1986.0,0,0.185565,0.247172,0.292067,-0.131803,POINT (77.73477 8.72260)
1,47,2891.0,2589.0,-0.194788,0.110169,train,1024.0,1236.0,1470.0,1742.0,1840.0,2005.0,-0.223704,1834.0,0,0.223704,0.305882,0.325843,-0.170699,POINT (77.73486 8.72260)
2,47,3066.0,2766.0,-0.185068,0.109269,train,1098.0,1288.0,1504.0,1878.0,1987.0,2097.0,-0.241547,1873.0,0,0.241547,0.346414,0.341794,-0.192498,POINT (77.73495 8.72260)
3,47,3066.0,2766.0,-0.198832,0.091151,train,1214.0,1372.0,1710.0,1878.0,1987.0,2097.0,-0.197890,2053.0,0,0.197890,0.267217,0.283920,-0.147956,POINT (77.73504 8.72260)
4,47,3034.0,2795.0,-0.224738,0.114013,train,1270.0,1404.0,1764.0,1877.0,1965.0,2146.0,-0.155369,2218.0,0,0.155369,0.346248,0.264694,-0.115101,POINT (77.73513 8.72260)


In [5]:
# Define the stratified sampling function
def stratified_sample(gdf, strata_column, n_samples_per_group, random_state=None):
    """
    Perform stratified sampling on a GeoDataFrame.

    Parameters:
    gdf (GeoDataFrame): The input GeoDataFrame.
    strata_column (str): The column name to stratify by.
    n_samples_per_group (int or dict): The number of samples to take per group.
                                       If an int is provided, it will be used for all groups.
                                       If a dict is provided, it should map group names to sample sizes.
    random_state (int, optional): Seed for the random number generator.

    Returns:
    GeoDataFrame: A stratified sample of the input GeoDataFrame.
    """
    if isinstance(n_samples_per_group, int):
        stratified_sample = gdf.groupby(strata_column).apply(lambda x: x.sample(n_samples_per_group, random_state=random_state)).reset_index(drop=True)
    elif isinstance(n_samples_per_group, dict):
        stratified_sample = gpd.GeoDataFrame(pd.concat(
            [x.sample(n_samples_per_group[group], random_state=random_state) for group, x in gdf.groupby(strata_column) if group in n_samples_per_group]
        ))
    else:
        raise ValueError("n_samples_per_group should be either an int or a dict")
    
    return stratified_sample

In [6]:
# Perform stratified sampling
n_samples_per_group = 50 # or a dictionary like {'group1': 5, 'group2': 15}
random_state = 42  # Set your random state for reproducibility
sampled_gdf = stratified_sample(gdf=gdf[gdf['sample'] == 'test'], strata_column='classvalue', n_samples_per_group=n_samples_per_group, random_state=45)

# Print the result
print(sampled_gdf.crs)
print(sampled_gdf.shape)
sampled_gdf.head()

GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433],AXIS["Longitude",EAST],AXIS["Latitude",NORTH]]
(300, 20)


,elevation,B11,B12,NDWI,NDVI,sample,B2,B3,B4,B5,B6,B7,NDMI,B8,classvalue,NDBI,EVI,NDBaI,NBR,geometry
0,48,3349.5,3110.0,-0.238340,0.100432,test,1050.0,1331.0,1769.0,2053.0,2110.0,2244.0,-0.215018,2164.0,0,0.215018,0.201366,0.308782,-0.179370,POINT (77.70989 8.70274)
1,17,2200.0,1998.0,-0.052259,-0.003356,test,1287.0,1605.0,1794.0,1849.0,1980.5,2007.5,-0.104972,1782.0,0,0.104972,-0.010364,0.101652,-0.057143,POINT (78.07954 8.82285)
2,9,3451.0,3190.0,-0.295381,0.158501,test,1078.0,1312.0,1752.0,2172.0,2394.0,2605.0,-0.177213,2412.0,0,0.177213,0.340909,0.326542,-0.138879,POINT (78.12212 8.56530)
3,26,1559.0,1462.0,-0.040117,-0.019931,test,1038.0,1316.0,1484.0,1441.0,1424.0,1486.0,-0.044556,1426.0,0,0.044556,-0.056952,0.024647,-0.012465,POINT (78.07766 8.82159)
4,51,3155.5,2866.5,-0.247472,0.110111,test,1001.0,1265.0,1681.0,1799.5,1879.0,1998.0,-0.201523,2097.0,0,0.201523,0.222389,0.304869,-0.155032,POINT (77.70710 8.70292)


In [7]:
# Load the classified image
tiff_path = os.path.join(output_dir, '03_CNN2019', 'lulc_2019.tif')
with rasterio.open(tiff_path) as src:
    classified_image = src.read(1)  # Read the first band
    transform = src.transform
    crs = src.crs
print(crs)

EPSG:4326


In [8]:
# Reproject the GeoDataFrame to the CRS of the classified image
sampled_gdf = sampled_gdf.to_crs(crs)
sampled_gdf.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- undefined
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [9]:
# Get ground truth labels and classified labels
ground_truth_labels = []
classified_labels = []

for idx, row in sampled_gdf.iterrows():
    geometry = row['geometry']
    class_value = row['classvalue']  # Adjust according to your attribute name in the shapefile

    # Create a mask for the current geometry
    mask = geometry_mask([geometry], transform=transform, invert=True, out_shape=classified_image.shape)

    # Extract the classified values within the mask
    classified_values = classified_image[mask]

    # Append the values to the lists
    ground_truth_labels.extend([class_value] * len(classified_values))
    classified_labels.extend(classified_values)

# Calculate the Kappa index
kappa = cohen_kappa_score(ground_truth_labels, classified_labels)
print(f'Kappa Index: {kappa}')

Kappa Index: 0.84


# kappa score 2023

In [21]:
# Load the shapefile
shapefile_path = os.path.join(rawDATA_dir, 'DLnew', 'extract_2023.shp')
gdf = gpd.read_file(shapefile_path)
print(gdf.shape)
gdf.head()

(215902, 20)


,elevation,B11,B12,NDWI,NDVI,sample,B2,B3,B4,B5,B6,B7,NDMI,B8,classvalue,NDBI,EVI,NDBaI,NBR,geometry
0,49,3216.0,2966.0,-0.215792,0.100343,train,1176.0,1450.0,1838.0,2010.0,2050.0,2193.0,-0.177160,2248.0,0,0.177160,0.229975,0.272655,-0.137706,POINT (77.70782 8.69924)
1,49,3167.0,2955.0,-0.172739,0.070079,train,1288.0,1578.0,1944.0,2113.0,2203.0,2358.0,-0.172095,2237.0,0,0.172095,0.172678,0.239288,-0.138290,POINT (77.70791 8.69924)
2,48,3167.0,2955.0,-0.153961,0.055606,train,1406.0,1698.0,2072.0,2113.0,2203.0,2358.0,-0.155207,2316.0,0,0.155207,0.145100,0.209009,-0.121229,POINT (77.70800 8.69924)
3,48,3152.0,2947.0,-0.216495,0.103890,train,1112.0,1444.0,1820.0,2134.0,2256.0,2423.0,-0.168706,2242.0,0,0.168706,0.218744,0.267900,-0.135864,POINT (77.70809 8.69924)
4,48,3152.0,2947.0,-0.229178,0.095053,train,1046.0,1416.0,1866.0,2134.0,2256.0,2423.0,-0.165250,2258.0,0,0.165250,0.174688,0.256277,-0.132373,POINT (77.70818 8.69924)


In [22]:
# Perform stratified sampling
n_samples_per_group = 50 # or a dictionary like {'group1': 5, 'group2': 15}
random_state = 42  # Set your random state for reproducibility
sampled_gdf = stratified_sample(gdf=gdf[gdf['sample'] == 'test'], strata_column='classvalue', n_samples_per_group=n_samples_per_group, random_state=45)

# Print the result
print(sampled_gdf.crs)
print(sampled_gdf.shape)
sampled_gdf.head()

GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433],AXIS["Longitude",EAST],AXIS["Latitude",NORTH]]
(250, 20)


,elevation,B11,B12,NDWI,NDVI,sample,B2,B3,B4,B5,B6,B7,NDMI,B8,classvalue,NDBI,EVI,NDBaI,NBR,geometry
0,12,3260.0,2962.0,-0.225775,0.084441,test,1322.0,1598.0,2136.0,2192.0,2226.0,2410.0,-0.126079,2530.0,0,0.126079,0.181333,0.208302,-0.078660,POINT (78.02358 8.62576)
1,46,3678.5,2775.5,-0.369314,0.210921,test,957.5,1338.0,1893.0,2308.0,2607.5,2797.0,-0.117491,2905.0,0,0.117491,0.357206,0.320470,0.022797,POINT (77.75911 8.78889)
2,13,3711.0,3509.0,-0.225026,0.130519,test,1296.0,1474.0,1792.0,2089.0,2143.0,2428.0,-0.228605,2330.0,0,0.228605,0.399941,0.348719,-0.201918,POINT (78.02340 8.62549)
3,41,2956.5,2462.5,-0.315749,0.195440,test,1072.0,1336.0,1729.0,1868.5,2290.5,2509.0,-0.070129,2569.0,0,0.070129,0.428222,0.261978,0.021167,POINT (77.75956 8.78781)
4,49,3397.5,2906.0,-0.245181,0.111696,test,1172.0,1537.0,2026.0,2365.5,2457.5,2625.0,-0.145289,2535.5,0,0.145289,0.215798,0.252881,-0.068088,POINT (77.75956 8.78952)


In [23]:
# Load the classified image
tiff_path = os.path.join(output_dir, '04_CNN2023new', 'lulc_2023.tif')
with rasterio.open(tiff_path) as src:
    classified_image = src.read(1)  # Read the first band
    transform = src.transform
    crs = src.crs
print(crs)

EPSG:4326


In [24]:
# Reproject the GeoDataFrame to the CRS of the classified image
sampled_gdf = sampled_gdf.to_crs(crs)
sampled_gdf.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- undefined
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [25]:
# Get ground truth labels and classified labels
ground_truth_labels = []
classified_labels = []

for idx, row in sampled_gdf.iterrows():
    geometry = row['geometry']
    class_value = row['classvalue']  # Adjust according to your attribute name in the shapefile

    # Create a mask for the current geometry
    mask = geometry_mask([geometry], transform=transform, invert=True, out_shape=classified_image.shape)

    # Extract the classified values within the mask
    classified_values = classified_image[mask]

    # Append the values to the lists
    ground_truth_labels.extend([class_value] * len(classified_values))
    classified_labels.extend(classified_values)

# Calculate the Kappa index
kappa = cohen_kappa_score(ground_truth_labels, classified_labels)
print(f'Kappa Index: {kappa}')

Kappa Index: 0.8001998001998002
